In [1]:
git_repo = '../../..'

import sys
import os

sys.path.insert(0, os.path.join(git_repo, 'BNP_sensitivity/GMM_clustering/'))

# Linear response libraries
import LinearResponseVariationalBayes as vb
import LinearResponseVariationalBayes.SparseObjectives as obj_lib

# My libraries
import gmm_clustering_lib as gmm_utils
import modeling_lib 
import functional_sensitivity_lib as fun_sens_lib 
import utils_lib

from scipy import spatial
import scipy.cluster.hierarchy as sch

import matplotlib.pyplot as plt
%matplotlib inline

import time
from tqdm import tqdm

from copy import deepcopy

import numpy as np
import scipy as sp

np.random.seed(453453)

import json 

In [2]:
iris_features, iris_species = utils_lib.load_data()
dim = iris_features.shape[1]

# Get some things that will be useful for plotting.
pca_fit, pc_features, colors1, colors2 = utils_lib.get_plot_data(iris_features)

# Set up the model

In [3]:
prior_params = gmm_utils.get_default_prior_params(dim)
prior_params['alpha'].set(8.0)
prior_params['prior_gamma_df'].set(8)
prior_params['prior_gamma_inv_scale'].set(np.eye(dim) * 0.62)

In [4]:
k_approx = 12
gh_deg = 8
model = gmm_utils.DPGaussianMixture(
    iris_features, k_approx, prior_params, gh_deg, use_logitnormal_sticks=True)

In [5]:
# run k-means init
n_kmeans_init = 50
init_global_free_param = model.cluster_and_set_inits(n_kmeans_init = n_kmeans_init)

In [6]:
t0 = time.time()
best_param, kl_hessian, kl_hessian_corrected, \
init_opt_time, newton_time, x_conv, f_conv, vb_opt = \
model.optimize_full(init_global_free_param,
    init_max_iter=100,
    final_max_iter=500)
t_newton = time.time() - t0

print('Done. ')

BGFS
Iter: 0	 RMSE: 5.4171664084976205	 Objective: 4927.074571257251
Iter: 10	 RMSE: 10.722601299848941	 Objective: 3164.887763988899
Iter: 20	 RMSE: 12.727784407538786	 Objective: 2095.64267039293
Iter: 30	 RMSE: 12.825320826126164	 Objective: 403.25309073085555
Iter: 40	 RMSE: 12.336111507461514	 Objective: -156.2818830513076
Iter: 50	 RMSE: 12.153510031465627	 Objective: -265.9051915783531
Iter: 60	 RMSE: 11.665404774476754	 Objective: -278.95947380457665
Iter: 70	 RMSE: 11.243659114866814	 Objective: -287.3217971112322
Iter: 80	 RMSE: 11.270215074059495	 Objective: -293.1724859597803
Iter: 90	 RMSE: 11.279599774958886	 Objective: -297.93642331028184
Iter: 100	 RMSE: 11.270952653637949	 Objective: -301.5500897148094
         Current function value: -302.542099
         Iterations: 100
         Function evaluations: 110
         Gradient evaluations: 110
Conditioned Newton:
i =  0
Iter: 110	 RMSE: 11.253624423017019	 Objective: -302.5420985282098
Iter: 120	 RMSE: 11.246263059285315	 

In [7]:
kl_hessian = model.objective.fun_free_hessian(best_param)

In [8]:
fit_dict = gmm_utils.get_checkpoint_dictionary(model, kl_hessian, seed=453453)
json_output_file = os.path.join(
    git_repo, 'BNP_sensitivity/GMM_clustering/iris_fits/ryan_iris_bnp_full_data_fit.json')
print('saving fit dict to ', json_output_file)
with open(json_output_file, 'w') as outfile:
    json.dump(fit_dict, outfile)


saving fit dict to  ../../../BNP_sensitivity/GMM_clustering/iris_fits/ryan_iris_bnp_full_data_fit.json
